In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
import sqlite3
import mysql.connector as cnt
import plotly.express as px

import datetime as dt

# import mysql.connector as cnt
import sqlite3

import requests
from bs4 import BeautifulSoup

# import selenium
from selenium.webdriver.chrome.options import Options
# from selenium.webdriver.firefox.options import Options as FirefoxOptions
# Firefox leistų sudėtingesnius nustatymus, bet neveikia per SNAP

from selenium import webdriver
from bs4 import BeautifulSoup
import time

print('Paskaita nr. 47, 2024-10-28')
print('Atverčia ir nuskaito AutoPlius skelbimus pagal anksčiau db išsaugotas nuorodas.')

# https://www.sqlitetutorial.net/sqlite-primary-key/
# https://www.crummy.com/software/BeautifulSoup/bs4/doc/
# https://www.sqlite.org/lang_createtable.html

db_file_name='auto_skelbimai_txt.db'
db_connect = sqlite3.connect(db_file_name)
cs = db_connect.cursor()
print(f"sqlite3.connect('{db_file_name}')")
print(dt.datetime.now().replace(microsecond=0))

Paskaita nr. 47, 2024-10-28
Atverčia ir nuskaito AutoPlius skelbimus pagal anksčiau db išsaugotas nuorodas.
sqlite3.connect('auto_skelbimai_txt.db')
2024-10-28 21:29:07


In [41]:
# https://stackoverflow.com/questions/38076220/python-mysqldb-connection-in-a-class
# https://stackoverflow.com/questions/75688682/how-should-i-type-my-sqlite-connection-and-cursor-in-a-python-class-before-i-pro
# https://pynative.com/python-sqlite/
# https://www.geeksforgeeks.org/python-sqlite-connecting-to-database/
# https://www.reddit.com/r/learnpython/comments/4lznux/best_technique_for_passing_sqlite3_connection/

import sqlite3

class things_with_properties:
    production=False # 'True' for production, 'False' for testing.
    class_type='things_with_properties'
    _current=False # Current instance id or False, if not set.

    def __init__(self,name): # constructor.
        self._name=name
        self._connection = sqlite3.connect(name)
        self._cursor = self._connection.cursor()
        if not self.production:
            print(f"Connected '{name}'\n")

    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        if not self.production:
            print(f"{self.class_type}.__exit__()")
        self.close() # .__exit__() vykdomas jei nebelieka kintamojo
    
    def __str__(self):
        if self._current:
            rez=f'Thing "{self._name}/{self._current}"\n'
            rez+=f'Parameters:"\n'
            rez+=f"{self._current}\n"
            return rez
        rez=f'The structure of database "{self._name}"\n'
        for (tableName,) in self.query(
        "select NAME from SQLITE_MASTER where TYPE='table' order by NAME;"
        ):
            rez+="{}:\n".format(tableName)
            for ( columnID,columnName,columnType,columnNotNull,columnDefault,columnPK,
            ) in self.query("pragma table_info('{}');".format(tableName)):
                rez+="  {id}: {name}({type}){null}{default}{pk}\n".format(
                    id=columnID,
                    name=columnName,
                    type=columnType,
                    null=" not null" if columnNotNull else "",
                    default=" [{}]".format(columnDefault) if columnDefault else "",
                    pk=" *{}".format(columnPK) if columnPK else "",
                )
        return rez

    @property
    def connection(self):
        return self._connection

    @property
    def cursor(self):
        return self._cursor

    def commit(self):
        self.connection.commit()

    def close(self, commit=True):
        if not self.production:
            print(f"{self.class_type}.close()")
        commit and self.commit()
        self.connection.close()

    def execute(self, sql, params=None): # result with .fetchall() or .fetchone()
        self._cursor.execute(sql, params or ())

    def fetchall(self):
        return self._cursor.fetchall()

    def fetchone(self):
        return self._cursor.fetchone()

    def query(self, sql, params=None):
        self._cursor.execute(sql, params or ())
        return self.fetchall()

    def current(self,id=0,reset=False):
        if reset:
            self._current=False
        elif id:
            self.execute('select id from object_x where id=? limit 1',(id,))
            if self.fetchone():
                self._current=id
            else:
                self._current=False
        return self._current

    def property(self,id=0,reset=False):
        if reset:
            self._current=False
            return False
        if id:
            self._current=id
        return self._current

if not things_with_properties.production:
    print("Class 'things_with_properties()' defined.")

Class 'things_with_properties()' defined.


In [45]:
db_objektas=things_with_properties(db_file_name)
db_objektas.execute('select * from object_x limit 1')
print(db_objektas.fetchone())
db_objektas.close()

Connected 'auto_skelbimai_txt.db'

(18267894, 'https://m.autoplius.lt/skelbimai/mercedes-benz-e250-2-5-l-sedanas-1990-dyzelinas-18267894.html', 1729622835, 1729622835)
things_with_properties.close()


In [46]:
db_objektas=things_with_properties(db_file_name)
# print(db_objektas)
print("Current instance:",db_objektas.current(18267894))
print()
print(db_objektas)
db_objektas.close()

Connected 'auto_skelbimai_txt.db'

Current instance: 18267894

Thing "auto_skelbimai_txt.db/18267894"
Parameters:"
18267894

things_with_properties.close()
